### YES24 크롤링

In [1]:
from webdriver_manager.chrome import ChromeDriverManager
ChromeDriverManager().install()

'/Users/mac/.wdm/drivers/chromedriver/mac64/121.0.6167.85/chromedriver-mac-x64/chromedriver'

In [2]:
from selenium import webdriver
browser = webdriver.Chrome()

In [3]:
# 3페이지까지 수집
# https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber=2&pageSize=24
from selenium.webdriver.common.by import By
import time
links = []
for i in range(1,4):
  print(f'{"*" * 10}현재 {i} 페이지 수집 중 입니다. {"*" * 10}')
  url = f'https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber={i}&pageSize=24'
  browser.get(url)
  
  time.sleep(1)
  datas = browser.find_elements(By.CLASS_NAME, 'gd_name')
  for data in datas:
    links.append(data.get_attribute('href'))
  time.sleep(2)

**********현재 1 페이지 수집 중 입니다. **********
**********현재 2 페이지 수집 중 입니다. **********
**********현재 3 페이지 수집 중 입니다. **********


In [4]:
# for link in links:
#   # 상세 페이지로 이동
#   browser.get(link)

browser.get(links[1])


In [10]:
# db 연동
import pymysql
import re
from datetime import datetime

conn = pymysql.connect(
  host='localhost',
  user='root',
  password='root',
  db='yes24',
  charset='utf8mb4',
  cursorclass=pymysql.cursors.DictCursor
)

with conn.cursor() as cur:
  for link in links:
    browser.get(link)
    title = browser.find_element(By.CLASS_NAME, 'gd_name').text
    # try:
    #   subtitle = browser.find_element(By.CLASS_NAME, 'gd_nameE').text
    # except:
    #   subtitle = None
    author = browser.find_element(By.CSS_SELECTOR, '.gd_auth a').text
    publisher = browser.find_element(By.CLASS_NAME, 'gd_pub').text
    publishing = browser.find_element(By.CLASS_NAME, 'gd_date').text

    match = re.search(r'(\d+)년 (\d+)월 (\d+)일', publishing)

    if match:
      year, month, day = match.groups()
      data_obj = datetime(int(year), int(month), int(day))
      publishing = data_obj.strftime("%Y-%m-%d")
    else:
      publishing = '2023-01-01'

    try:
      rating = browser.find_element(By.CSS_SELECTOR, '.gd_rating .yes_b').text
    except:
      rating = '0'
      
    try:
      review = browser.find_element(By.CSS_SELECTOR, '.gd_reviewCount .txC_blue').text
      review = int(review.replace(',',''))
    except:
      review = 0

    sales = browser.find_element(By.CLASS_NAME, 'gd_sellNum').text.split(' ')[2]
    sales = int(sales.replace(',',''))
    
    price = browser.find_element(By.CSS_SELECTOR, '.nor_price .yes_m').text
    price = int(price.replace(',',''))

    full_text = browser.find_element(By.CSS_SELECTOR, '.gd_best dd').text
    parts = full_text.split(' | ')
    ranking = 0
    ranking_weeks = 0
    if len(parts) > 1:
      try:
        ranking_part = parts[0]
        ranking = ''.join(filter(str.isdigit, ranking_part))
      except:
        pass

      try:
        ranking_weeks_part = parts[1]
        ranking_weeks = ''.join(filter(str.isdigit, ranking_weeks_part.split()[-1]))
      except:
        pass
    sql = '''INSERT INTO Books(
      title, author, publisher, publishing, rating, review, sales, price, ranking, ranking_weeks)
      VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
      '''
    
    cur.execute(sql, (title, author, publisher, publishing, rating, review, sales, price, ranking, ranking_weeks))
    conn.commit()
    time.sleep(2)
  

None <class 'NoneType'>
